Lib imports

In [1]:
from gx_utils import *
import logging
import numpy as np
import random
from random import seed, choice
from typing import Callable

logging.basicConfig(format="%(message)s", level=logging.INFO)

Problem definition - lab delivery

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

Professor's solution - lab delivery

In [ ]:
def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    print(all_lists, " all lists")
    while goal != covered:
        x = all_lists.pop(0)
        print(x, " x")
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")
greedy(5)

Greedy solution

In [ ]:
def set_covering_problem_greedy(universe,subsets,costs):
    cost=0
    visited=0
    #create a unique set - source: documentation
    elements=set(e for s in subsets for e in s) 
    if elements!=universe:
        return None
    covered=set()
    solution=[]
    while covered!=elements:
        subset = max(subsets,key=lambda s: len(s-covered)/costs[subsets.index(s)])
        solution.append(subset)
        cost+=costs[subsets.index(subset)]
        visited=visited+1
        covered |= subset
    print("NUMBER OF VISITED NODES: ",visited)
    print("w: ",sum(len(_) for _ in solution))
    return solution,cost 

def start(N):
    """ Start: 
        N: N
        I suppose cost are unit
    """
    sub=problem(N,seed=42)
    #all costs are 1 
    C=np.full(len(sub),1)
    #print("C ",C) #all 1
    universe=set(range(N))
    #create a unique set - source: documentation
    subsets=[set(element) for element in sub]
    costs=C 
    set_cover,totalCosts=set_covering_problem_greedy(universe,subsets,costs)
    print("UNIVERSE: ",universe)
    print("COVERING SETS: ",set_cover)
    print("COST: ",totalCosts)



#N=5
print("5")
start(5)
print("----------------------------------------------------------------")
print("10")
start(10)
print("----------------------------------------------------------------")
print("20")
start(20)
print("----------------------------------------------------------------")
print("50")
start(50)
print("----------------------------------------------------------------")
print("100")
start(100)
print("----------------------------------------------------------------")
print("500")
start(500)
print("----------------------------------------------------------------")
print("1000")
start(1000)



Breadth first solution

In [ ]:
def set_covering_problem_bf(universe,subsets,costs):
    cost=0
    elements=set(e for s in subsets for e in s)
    if elements!=universe:
        print("The subsets don't contain the universe.")
        return None
    covered=set()
    solution=[]
    queue=[]
    visited=[]
    while covered!=elements:
        subset = max(subsets,key=lambda s: len(s-covered))
        #print(subset, " subset")
        queue.append(subset)
        x=queue.pop()
        if x not in visited:
            visited.append(subset)
            solution.append(subset)
            cost+=costs[subsets.index(subset)]
            covered |= subset
    print("NUMBER OF VISITED NODES: ",len(visited))
    print("w: ",sum(len(_) for _ in solution))
    return solution,cost

def start(N):
    """ Start: 
        N: N
        I suppose cost are unit
    """
    sub=problem(N,seed=42)
    #all costs are 1 
    C=np.full(len(sub),1)
    #print("C ",C)
    universe=set(range(N))
    #create a unique set - source: documentation
    subsets=[set(element) for element in sub]
    costs=C 
    set_cover,totalCosts=set_covering_problem_bf(universe,subsets,costs)
    print("UNIVERSE: ",universe)
    print("COVERING SETS: ",set_cover)
    print("COST: ",totalCosts)


#N=5
print("5")
start(5)
print("----------------------------------------------------------------")
print("10")
start(10)
print("----------------------------------------------------------------")
print("20")
start(20)
print("----------------------------------------------------------------")
print("50")
start(50)
print("----------------------------------------------------------------")
print("100")
start(100)
print("----------------------------------------------------------------")
print("500")
start(500)
print("----------------------------------------------------------------")
print("1000")
start(1000)


A* Searching

In [3]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [9]:
#GOAL = State(np.array(list(range(1, SIZE**2)) + [0]).reshape((SIZE, SIZE)))
#logging.info(f"Goal:\n{GOAL}")

#true is goal is reached
def goal_test(state,N):
    GOAL = list(set(range(N)))
    state_set=set()
    test=state.copy_data().tolist()
    logging.info(f"Goal:\n{GOAL}")
    for e in test:
        [state_set.add(i) for i in e]
    return list(state_set)==GOAL

In [11]:
#return a new_state if elem contains element different from state
def result(state, elem):
    elem_set = set(elem)
    state_set = set()
    for e in state.copy_data():
        [state_set.add(i) for i in e]
    intersection = list(elem_set - state_set)
    if intersection == []: return False

    new_state = []
    [new_state.append(e) for e in state.copy_data().tolist()]
    new_state.append(elem)
    print(f'append element {elem}')
    new_state = sorted(new_state, key=lambda l: len(l))
    
    return State(np.array(new_state))

In [13]:
input_state=sorted(problem(100,seed=42),key=lambda l:len(l))
INITIAL_STATE=State(np.array([input_state[0]]))

In [7]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
    N
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0
    state_repeat=0
    GOAL=list(set(range(N)))
    
    for elem in input_state:
        while state is not None and not goal_test(state,N):
            print(f'n.state {state_repeat}, state: {state}, elem: {elem}')
            #index = input_state.index(elem)
            new_state = result(state, elem)
            # check if new node contains element different from ones in the state
            if not new_state or elem in state.copy_data().tolist():
                break
            else:
                cost = 1
                if new_state not in state_cost and new_state not in frontier:
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
                    frontier.push(new_state, p=priority_function(new_state))
                    logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
                elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                    old_cost = state_cost[new_state]
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
                    logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        
            if frontier:
                state = frontier.pop()
            else:
                state = None
        
    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))

In [12]:
parent_state = dict()
state_cost = dict()


def h(state):
    return np.sum((state._data != GOAL._data) & (state._data > 0))

#A star
for n in [5,10]:
    final = search(
        INITIAL_STATE,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: state_cost[s] + h(s),
        unit_cost=lambda a: len(a),
        N=n
    )

NameError: name 'GOAL' is not defined